In [13]:
import pandas as pd
import datetime

#Load the game data to pandas data frame
df_season_map  = pd.read_csv('../data/game_dates_season.csv')
df_raw = pd.read_csv('../data/nfl_games_pfr_raw.csv')
df_teams = pd.read_csv('../data/teams.csv')

In [14]:
print(df_raw.head())

   Week  Day        Date    Time         Winner/tie Unnamed: 5  \
0     5  Thu  2024-10-03  8:15PM    Atlanta Falcons        NaN   
1     5  Sun  2024-10-06  9:30AM      New York Jets          @   
2     5  Sun  2024-10-06  1:00PM      Buffalo Bills          @   
3     5  Sun  2024-10-06  1:00PM  Carolina Panthers          @   
4     5  Sun  2024-10-06  1:00PM   Cleveland Browns          @   

               Loser/tie Unnamed: 7  PtsW  PtsL   YdsW  TOW   YdsL  TOL  
0   Tampa Bay Buccaneers   boxscore  36.0  30.0  550.0  1.0  333.0  1.0  
1      Minnesota Vikings    preview   NaN   NaN    NaN  NaN    NaN  NaN  
2         Houston Texans    preview   NaN   NaN    NaN  NaN    NaN  NaN  
3          Chicago Bears    preview   NaN   NaN    NaN  NaN    NaN  NaN  
4  Washington Commanders    preview   NaN   NaN    NaN  NaN    NaN  NaN  


In [15]:
#Columns: Week,Day,Date,Time,Winner/tie,Unnamed: 5,Loser/tie,,PtsW,PtsL,YdsW,TOW,YdsL,TOL
df_games = df_raw
df_teams_abbr = df_teams[['Team Name','Team Abbr']]

df_games = pd.merge(df_games, df_season_map, on='Date', how='left')
df_games = pd.merge(df_games, df_teams_abbr, left_on='Winner/tie', right_on='Team Name', how='left')
df_games = pd.merge(df_games, df_teams_abbr, left_on='Loser/tie', right_on='Team Name', how='left')

if len(df_games[df_games['season'].isna()])==0:
    df_games[['season','week']] = df_games[['season','week']].round().astype(int)
    
else:
    print("Some game dates not included in mapping file")
    print(df_games[df_games['season'].isna()])

df_games = df_games.drop(columns=['Unnamed: 7','Unnamed: 0','Team Name_x','Team Name_y']).rename(columns={'Unnamed: 5':'LoserIsHome', 'Winner/tie':'Winner','Loser/tie':'Loser', 'Team Abbr_x':'Winner Abbr','Team Abbr_y':'Loser Abbr'})
df_games['Home Team'] = df_games.apply(lambda row: row['Loser Abbr'] if row['LoserIsHome']=="@" else row['Winner Abbr'], axis=1)
df_games['Away Team'] = df_games.apply(lambda row: row['Winner Abbr'] if row['LoserIsHome']=="@" else row['Loser Abbr'], axis=1)
df_games['Winner Yds'] = df_games.apply(lambda row: row['Winner Abbr'] if row['YdsW']>=row['YdsL'] else row['Loser Abbr'], axis=1)
df_games['Loser Yds'] = df_games.apply(lambda row: row['Winner Abbr'] if row['YdsW']<row['YdsL'] else row['Loser Abbr'], axis=1)
df_games['Margin Yds'] = df_games.apply(lambda row: abs(row['YdsW']-row['YdsL']), axis=1)
df_games['Winner TO'] = df_games.apply(lambda row: row['Winner Abbr'] if row['TOW']>=row['TOL'] else row['Loser Abbr'], axis=1)
df_games['Loser TO'] = df_games.apply(lambda row: row['Winner Abbr'] if row['TOW']<row['TOL'] else row['Loser Abbr'], axis=1)
df_games['Margin TO'] = df_games.apply(lambda row: abs(row['TOW']-row['TOL']), axis=1)
df_games['Margin Pts'] = df_games.apply(lambda row: abs(row['PtsW']-row['PtsL']), axis=1)
df_games['Total Pts'] = df_games.apply(lambda row: abs(row['PtsW']+row['PtsL']), axis=1)
df_games['Margin Pct'] = df_games.apply(lambda row: row['Margin Pts']-row['Total Pts'], axis=1)
df_games['home_team_winner'] = df_games.apply(lambda row: False if row['LoserIsHome']=="@" else True, axis=1)
df_games['away_win_bonus'] = df_games.apply(lambda row: 0 if row['home_team_winner'] else 1, axis=1)

print(df_games.head())

#print(df_games[df_games['Team Abbr_x'].isna()])
### Need to add error handling for unmapped abbreviations

   Week  Day        Date    Time             Winner LoserIsHome  \
0     5  Thu  2024-10-03  8:15PM    Atlanta Falcons         NaN   
1     5  Sun  2024-10-06  9:30AM      New York Jets           @   
2     5  Sun  2024-10-06  1:00PM      Buffalo Bills           @   
3     5  Sun  2024-10-06  1:00PM  Carolina Panthers           @   
4     5  Sun  2024-10-06  1:00PM   Cleveland Browns           @   

                   Loser  PtsW  PtsL   YdsW  ...  Loser Yds  Margin Yds  \
0   Tampa Bay Buccaneers  36.0  30.0  550.0  ...         TB       217.0   
1      Minnesota Vikings   NaN   NaN    NaN  ...        MIN         NaN   
2         Houston Texans   NaN   NaN    NaN  ...        HOU         NaN   
3          Chicago Bears   NaN   NaN    NaN  ...        CHI         NaN   
4  Washington Commanders   NaN   NaN    NaN  ...        WAS         NaN   

   Winner TO  Loser TO  Margin TO Margin Pts Total Pts Margin Pct  \
0        ATL        TB        0.0        6.0      66.0      -60.0   
1       

In [16]:
df_games.to_csv('../data/nfl_games_pfr.csv')